<a href="https://colab.research.google.com/github/Cris-0506/Cris-0506/blob/main/Projeto_Enem_2019_SP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Projeto Enem 2019 - São Paulo /
Dados disponíveis no site do Inep (http://inep.gov.br/microdados)

Objetivo: Analisar o desempenho entre os vestibulandos de escolas públicas e privadas do Estado de São Paulo classificados na prova do Enem de 2019. 

In [1]:
import numpy as np
import pandas as pd


In [2]:
dados = pd.read_csv('/content/microdados_enem_2019_sp.csv', sep=';', encoding = 'iso-8859-1')

In [ ]:
dados.head()

In [4]:
dados.shape

(70833, 44)

In [ ]:
dados.dtypes

In [6]:
# Removendo colunas com dados que não serão utilizados na análise.

dados1 = dados.drop(columns = ['NU_INSCRICAO', 'CO_UF_RESIDENCIA','SG_UF_RESIDENCIA', 'TP_ESTADO_CIVIL'])

In [7]:
dados1 = dados1.drop(columns = ['TP_NACIONALIDADE','CO_MUNICIPIO_NASCIMENTO', 'NO_MUNICIPIO_NASCIMENTO', 'CO_UF_NASCIMENTO', 'SG_UF_NASCIMENTO'])

In [8]:
dados1 = dados1.drop(columns = ['TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ENSINO'])

In [9]:
dados1.shape

(70833, 32)

In [10]:
# Renomeando as variáveis

dados1 = dados1.rename (columns = {'NU_IDADE':'IDADE', 'TP_SEXO':'SEXO', 'TP_COR_RACA':'RACA','Q025': 'INTERNET'})

In [11]:
dados1 = dados1.rename (columns = {'NU_NOTA_REDACAO': 'NOTA_REDACAO', 'NU_NOTA_CN': 'NOTA_CN', 'NU_NOTA_CH': 'NOTA_CH', 'NU_NOTA_LC':'NOTA_LC', 'NU_NOTA_MT': 'NOTA_MT'})

In [12]:
dados1 = dados1.rename (columns = {'TP_ESCOLA':'ESCOLA', 'NU_NOTA_COMP1':'COMP1', 'NU_NOTA_COMP2': 'COMP2', 'NU_NOTA_COMP3': 'COMP3', 'NU_NOTA_COMP4': 'COMP4', 'NU_NOTA_COMP5': 'COMP5'})

In [ ]:
dados1.head()

In [14]:
# A soma das notas deve dar no máximo 1000, mas algumas variáveis estão com as notas multiplicadas por 10 (MT, CL, CH e CN) e dessa forma necessitam de correção.

dados1.loc [:,'NOTA_CN']/=10


In [15]:
dados1.loc [:,'NOTA_CH']/=10


In [16]:
dados1.loc [:,'NOTA_MT']/=10


In [17]:
dados1.loc [:,'NOTA_LC']/=10

In [ ]:
dados1.head()

In [19]:
# Os registros de algumas variáveis qualitativas estão na forma de códigos numéricos, é necessário renomear esses registros.

dados1['RACA']= dados1['RACA'].replace ({0:'Não declarado', 1: 'Branca', 2: 'Preta', 3: 'Parda', 4: 'Amarela', 5: 'Indígena'})

In [20]:
dados1['ESCOLA'] = dados1['ESCOLA'].replace({1:'Nao respondeu', 2: 'Pública', 3: 'Privada', 4:'Exterior'})

In [21]:
dados1['INTERNET'] = dados1['INTERNET'].replace({'A': 'Não', 'B': 'Sim'})

In [ ]:
dados1['IDADE'].value_counts()

In [23]:
# O Enem pode ser feito por alunos do primeiro e segundo colegial como treineiros, porém serão utilizados somente os dados dos vestibulandos, ou seja, acima de 18 anos. 
vestibulandos = dados1.loc[dados1.IDADE>18]


In [ ]:
vestibulandos.head()

In [25]:
# Averiguando se há algum treineiro acima de 18 anos. Registro: 1 (é treineiro) e 0 (não é treineiro) 
vestibulandos.query('IN_TREINEIRO == 1')['IN_TREINEIRO'].value_counts()

1    68
Name: IN_TREINEIRO, dtype: int64

In [26]:
# Excluindo os treineiros do DataFrame vestibulandos.
vestibulandos = vestibulandos.loc[vestibulandos.IN_TREINEIRO == 0]

In [27]:
vestibulandos.shape

(31199, 32)

In [28]:
# A análise será feita utilizando somente os dados dos vestibulandos classificados. As provas LC, CH e redação foram feitas no mesmo dia, CN e MT em outro dia.
# Assim, o aluno perde mais de uma prova caso falte um dia.
# Registros: 0 = faltou; 1 = classificado; 2 = desclassificado
classificados = vestibulandos.loc[vestibulandos.TP_PRESENCA_CH == 1]


In [29]:
classificados = classificados.loc[classificados.TP_PRESENCA_MT == 1]

In [ ]:
classificados.isnull().sum()

In [31]:
# Removendo demais colunas que não serão utilizadas nas análises
classificados = classificados.drop(columns = ['CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC'])


In [ ]:
classificados.head()

In [33]:
classificados.shape

(18989, 25)

In [34]:
from IPython.utils.sysinfo import encoding
classificados.to_csv('enem_sp_2019.csv', sep = ';', encoding = 'iso-8859-1', index=False)

Amostragem aleatória simples

In [35]:
enem_sp = pd.read_csv('/content/enem_sp_2019.csv', sep = ';', encoding = 'iso-8859-1')

In [36]:
np.random.seed(1)

In [37]:
amos_simples = enem_sp.sample (3500)

In [38]:
amos_simples.shape

(3500, 25)

In [39]:
dados_enem = amos_simples

In [ ]:
dados_enem.describe

In [41]:
dados_enem = dados_enem.drop(columns=['CO_ESCOLA'])

In [ ]:
dados_enem.dtypes

1. Análise da Nota de Redação

In [43]:
import plotly.express as px

In [44]:
import seaborn as sns

In [45]:
import scipy.stats as stats
import matplotlib.pyplot as plt

In [46]:
escola_publica = dados_enem.loc[dados_enem.ESCOLA=='Pública']

In [47]:
escola_publica.shape

(165, 24)

In [48]:
escola_privada = dados_enem.loc[dados_enem.ESCOLA=='Privada']

In [49]:
escola_privada.shape

(29, 24)

In [50]:
stats.shapiro(escola_publica.NOTA_REDACAO)

ShapiroResult(statistic=0.9086031913757324, pvalue=1.2533441839934767e-08)

In [51]:
stats.shapiro(escola_privada.NOTA_REDACAO)

ShapiroResult(statistic=0.8670300245285034, pvalue=0.0017389527056366205)

In [52]:
# A distribuição das médias das escolas públicas não é normal, dessa maneira é necessário usar um teste não-paramétrico. 
# Observando esses resultados em histograma e gráfico QQ Plot.

In [ ]:
stats.probplot(escola_publica['NOTA_REDACAO'], dist='norm', plot=plt)
plt.title('Normal QQ Plot - Escola Pública')
plt.show()

In [ ]:
stats.probplot(escola_privada['NOTA_REDACAO'], dist='norm', plot=plt)
plt.title('Normal QQ Plot - Escola Privada')
plt.show()

In [ ]:
from seaborn.distributions import histplot
sns.histplot(escola_publica, x='NOTA_REDACAO', bins=10, color='orange', kde=True, stat='probability')

In [ ]:
sns.histplot(escola_privada, x='NOTA_REDACAO', bins=10, color='orange', kde=True, stat='probability')

Objetivo: analisar as medianas das notas de redação das escolas públicas e privadas.
*  H0: não há diferença entre as medianas (p>0,05). 
*  Ha: há diferença entre as medianas (p<=0,05).



In [57]:
# Teste não paramétrico de Mann-Whitney
stat, p = stats.mannwhitneyu(escola_publica.NOTA_REDACAO, escola_privada.NOTA_REDACAO)
print('Estatística do Teste: {}:'.format(stat.round(2)))
print('p-valor:{}'.format(p))

Estatística do Teste: 1855.0:
p-valor:0.05372625743170889


In [ ]:
escola_publica.median()

In [ ]:
escola_privada.median()

Conclusão: Considerando o intervalo de confiança de 95%, não podemos concluir que exista diferença entre as medianas das notas de redação entre escolas públicas e privadas.
Não rejeita H0.


*   Mediana escola privada = 560
*   Mediana escola pública = 520



2. Análise da Nota de Matemática

In [60]:
stats.shapiro(escola_publica.NOTA_MT)

ShapiroResult(statistic=0.85226970911026, pvalue=1.307145277434918e-11)

In [61]:
stats.shapiro(escola_privada.NOTA_MT)

ShapiroResult(statistic=0.9106526374816895, pvalue=0.017793038859963417)

In [62]:
# A distribuição das notas matemática de escolas públicas não é normal, dessa forma será necessário um teste não paramétrico.
# Observação dos dados nos gráficos QQ Plot e histograma.

In [ ]:
stats.probplot(escola_publica['NOTA_MT'], dist='norm', plot=plt)
plt.title('Normal QQ Plot - Escola Pública - Matemática')
plt.show()

In [ ]:
stats.probplot(escola_privada['NOTA_MT'], dist='norm', plot=plt)
plt.title('Normal QQ Plot - Escola Privada - Matemática')
plt.show()

In [ ]:
sns.histplot(escola_privada, x='NOTA_MT', bins=10, color='orange', kde=True, stat='probability')

In [ ]:
sns.histplot(escola_privada, x='NOTA_MT', bins=10, color='orange', kde=True, stat='probability')

Objetivo: Analisar se há diferença nas medianas das notas de matemática entre escolas públicas e privadas.


*   H0: Não existe diferença (p>0,05)
*   Ha: Existe diferença (p<=0,05)



In [67]:
stat, p=stats.mannwhitneyu(escola_publica.NOTA_MT, escola_privada.NOTA_MT)
print('Estatística do Teste: {}'.format(stat.round(2)))
print('p-valor:{}'.format(p))

Estatística do Teste: 1982.0
p-valor:0.14147064021214475


In [ ]:
escola_publica.median()

In [ ]:
escola_privada.median()

Conclusão: Considerando o intervalo de confiança de 95%, podemos concluir que existe diferença entre as medianas das notas de matemática entre escolas públicas e privadas. Rejeita H0. O desempenho dos alunos de escolas privadas na prova de matemática foi superior ao de escolas públicas.
 

*   Mediana escola privada = 538,8
*   Mediana escola pública = 455,3



3. Análise da Nota de Ciências Naturais

In [70]:
stats.shapiro(escola_publica.NOTA_CN)

ShapiroResult(statistic=0.772907555103302, pvalue=1.0333017603993307e-14)

In [71]:
stats.shapiro(escola_privada.NOTA_CN)

ShapiroResult(statistic=0.7317918539047241, pvalue=6.219795523065841e-06)

In [72]:
# A distribuição das notas ciências naturais de escolas públicas não é normal, dessa forma será necessário um teste não paramétrico.
# Observação dos dados nos gráficos QQ Plot e histograma.

In [ ]:
stats.probplot(escola_publica['NOTA_CN'], dist='norm', plot=plt)
plt.title('Normal QQ Plot - Escola Pública - Ciências Naturais')
plt.show()

In [ ]:
stats.probplot(escola_privada['NOTA_CN'], dist='norm', plot=plt)
plt.title('Normal QQ Plot - Escola Privada - Ciências Naturais')
plt.show()

In [ ]:
sns.histplot(escola_publica, x='NOTA_CN', bins=10, color='purple', kde=True, stat='probability')

In [ ]:
sns.histplot(escola_privada, x='NOTA_CN', bins=10, color='purple', kde=True, stat='probability')

Objetivo: Analisar se há diferença nas medianas das notas de ciências naturais entre escolas públicas e privadas.

*   H0: Não existe diferença (p>0,05)
*   Ha: Existe diferença (p<=0,05)

In [77]:
stat, p=stats.mannwhitneyu(escola_publica.NOTA_CN, escola_privada.NOTA_CN)
print('Estatística do Teste: {}'.format(stat.round(2)))
print('p-valor:{}'.format(p))

Estatística do Teste: 2096.5
p-valor:0.2892716606266096


In [ ]:
escola_publica.median()

In [ ]:
escola_privada.median()

Conclusão: Considerando o intervalo de confiança de 95%,podemos concluir que existe diferença entre as medianas das notas de ciências naturais entre escolas públicas e privadas. O desempenho dos alunos de escola privada foi melhor que dos alunos de escola pública.Rejeita H0.

*  Mediana escola privada = 503,70
*  Mediana escola pública = 433,20 


4. Análise da Nota de Linguagem e Códigos 

In [80]:
stats.shapiro(escola_publica.NOTA_LC)

ShapiroResult(statistic=0.6691642999649048, pvalue=9.184708292142787e-18)

In [81]:
stats.shapiro(escola_privada.NOTA_LC)

ShapiroResult(statistic=0.6838060617446899, pvalue=1.2425335853549768e-06)

In [ ]:
stats.probplot(escola_publica['NOTA_LC'], dist='norm', plot=plt)
plt.title('Normal QQ Plot - Escola Pública - Linguagens e Códigos')
plt.show()

In [ ]:
stats.probplot(escola_privada['NOTA_LC'], dist='norm', plot=plt)
plt.title('Normal QQ Plot - Escola Privada - Linguagens e Códigos')
plt.show()

In [ ]:
sns.histplot(escola_publica, x='NOTA_LC', bins=10, color='pink', kde=True, stat='probability')

In [ ]:
sns.histplot(escola_privada, x='NOTA_LC', bins=10, color='pink', kde=True, stat='probability')

Objetivo: Analisar se há diferença nas medianas das notas de linguagens e códigos entre escolas públicas e privadas.


*   H0: Não existe diferença (p>0,05)
*   Ha: Existe diferença (p<=0,05)


In [86]:
stat, p=stats.mannwhitneyu(escola_publica.NOTA_LC, escola_privada.NOTA_LC)
print('Estatística do Teste: {}'.format(stat.round(2)))
print('p-valor:{}'.format(p))

Estatística do Teste: 2184.5
p-valor:0.4567950284713853


Conclusão: Considerando o intervalo de confiança de 95%, não podemos concluir que exista diferença entre as medianas das notas de linguagens e códigos entre escolas públicas e privadas.
Não rejeita H0.


*   Mediana escola privada = 520,70
*   Mediana escola publica = 507,35


5. Análise da nota de Ciências Humanas

In [87]:
stats.shapiro(escola_publica.NOTA_CH)

ShapiroResult(statistic=0.7907549142837524, pvalue=4.328378001611406e-14)

In [88]:
stats.shapiro(escola_privada.NOTA_CH)

ShapiroResult(statistic=0.7545619606971741, pvalue=1.4153321899357252e-05)

In [ ]:
stats.probplot(escola_publica['NOTA_CH'], dist='norm', plot=plt)
plt.title('Normal QQ Plot - Escola Pública - Ciências Humanas')
plt.show()

In [ ]:
stats.probplot(escola_privada['NOTA_CH'], dist='norm', plot=plt)
plt.title('Normal QQ Plot - Escola Privada - Ciências Humanas')
plt.show()

In [ ]:
sns.histplot(escola_publica, x='NOTA_CH', bins=10, color='red', kde=True, stat='probability')

In [ ]:
sns.histplot(escola_privada, x='NOTA_CH', bins=10, color='red', kde=True, stat='probability')

Objetivo: Analisar se há diferença nas medianas das notas de ciências humanas entre escolas públicas e privadas.


*   H0: Não existe diferença (p>0,05)
*   Ha: Existe diferença (p<=0,05)

In [93]:
stat, p=stats.mannwhitneyu(escola_publica.NOTA_CH, escola_privada.NOTA_CH)
print('Estatística do Teste: {}'.format(stat.round(2)))
print('p-valor:{}'.format(p))

Estatística do Teste: 2319.0
p-valor:0.7934818507423063


Conclusão: Considerando o intervalo de confiança de 95%, não podemos concluir que exista diferença entre as medianas das notas de ciências humanas entre escolas públicas e privadas.
Não rejeita H0.


*   Mediana escola privada = 494,40
*   Mediana escola pública = 472,20


Conclusão do Projeto

Os alunos de escolas privadas foram superiores em desempenho nas provas de Ciências Naturais e Matemática, embora não tenha diferença entre as demais provas. Dessa forma, conclui-se que para que os alunos de escolas públicas possam competir de forma igualitária nas provas do Enem é necessário maior atenção da parte dos governantes à educação no Estado de São Paulo.